In [1]:
!pip install plotly_express
!wget -O tracelib.py https://raw.githubusercontent.com/HiEST/bojos/master/traces/tracelib.py
!wget -O emissions_eq.py https://raw.githubusercontent.com/HiEST/bojos/master/extras/emissions_eq.py

--2024-04-27 16:31:40--  https://raw.githubusercontent.com/HiEST/bojos/master/traces/tracelib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1113 (1.1K) [text/plain]
Saving to: ‘tracelib.py’

tracelib.py         100%[===================>]   1.09K  --.-KB/s    in 0s      

2024-04-27 16:31:40 (35.3 MB/s) - ‘tracelib.py’ saved [1113/1113]

--2024-04-27 16:31:40--  https://raw.githubusercontent.com/HiEST/bojos/master/extras/emissions_eq.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5470 (5.3K) [text/plain]
Saving to: ‘emissi

In [2]:
import datetime
from emissions_eq import *
from google.colab import files
from tracelib import *
import pandas as pd
import numpy as np
import math

In [3]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]
traces = pd.read_csv(filename)

Saving data.csv to data.csv


In [5]:
traces['nox'] = traces.apply(lambda row: row.noxme + row.noxae, axis=1)
traces['sox'] = traces.apply(lambda row: row.soxme + row.soxae, axis=1)
traces['co2'] = traces.apply(lambda row: row.co2me + row.co2ae, axis=1)
traces['co'] = traces.apply(lambda row: row.come + row.coae, axis=1)

Cast ts a datetime (tipo de datos consciente del tiempo)

In [6]:
traces['timestamp'] = pd.to_datetime(traces['timestamp'])

# Acumulado de cada traza

In [7]:
traces.drop(columns = ['timestamp', 'uid', 'gid', 'lat', 'lon']).groupby('sid').sum()

,noxme,noxae,soxme,soxae,co2me,co2ae,come,coae,nox,sox,co2,co
sid,,,,,,,,,,,,
1,1751.938010,1122.492662,53.914886,34.544067,84051.652270,53853.139966,3.756276e+08,0,2874.430672,88.458953,137904.792236,3.756276e+08
3,858.397632,1073.152325,26.416694,33.025647,41182.815164,51485.968978,3.620459e+08,0,1919.214873,59.062736,92076.991396,3.620459e+08
4,485.756733,234.366600,14.948885,7.212498,23304.851957,11244.062191,2.461744e+09,0,720.123332,22.161382,34548.914148,2.461744e+09


# Traza 1

In [8]:
trace1 = traces[traces.sid == 1]

In [9]:
pollutant = "co2" # nox, sox, co2, co
plot_trace(trace1, height = 600, pollutant = pollutant)
plot_trace_line(trace1, height = 600)

# Speed + Acceleration

In [10]:
speed = calculateSpeedAccPandas(trace1)

In [11]:
plot_line(speed, 'id', 'speed', title="Speed")
plot_line(speed, 'id', 'acc', title="Acceleration")
plot_line(speed, 'id', 'timeDiff', title="Time between messages")

# Emissions

In [12]:
ship = getShipProfile()

In [13]:
ship['type'] # Try changing this value to "Yatch" and take a look to the auxiliary engine!

'Passenger/Ro-Ro Cargo Ship'

In [14]:
emis = computeEmissions(ship, speed['speed'], trace1['timestamp'])

In [15]:
emis.drop(columns = ['id']).sum()


speed    3.738521e+03
noxme    1.557991e+03
soxme    4.794626e+01
co2me    7.474675e+04
come     8.052885e+06
noxae    1.122493e+03
soxae    3.454407e+01
co2ae    5.385314e+04
coae     0.000000e+00
pme      4.319652e+05
pae      3.112200e+05
dtype: float64

In [16]:
plot_line(emis, 'id', 'soxme', title='SOx ME')
plot_line(emis, 'id', 'soxae', title='SOx AE')
plot_line(emis, 'id', 'noxme', title='NOx ME')
plot_line(emis, 'id', 'noxae', title='NOx AE')
plot_line(emis, 'id', 'co2me', title='CO2 ME')
plot_line(emis, 'id', 'co2ae', title='CO2 AE')
plot_line(emis, 'id', 'come', title='CO ME')


In [17]:
emis

,id,speed,noxme,soxme,co2me,come,noxae,soxae,co2ae,coae,pme,pae
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,12.335084,0.379605,591.792747,0,0.000000,3420
1,1,88.386350,48.476608,1.491840,2325.732385,79040.644258,12.335084,0.379605,591.792747,0,13440.524236,3420
2,2,-994.882620,0.000000,0.000000,0.000000,0.000000,12.335084,0.379605,591.792747,0,0.000000,3420
3,3,24.930190,48.476608,1.491840,2325.732385,182396.176305,12.335084,0.379605,591.792747,0,13440.524236,3420
4,4,3.945277,0.303755,0.009348,14.573069,19.598033,12.335084,0.379605,591.792747,0,84.218498,3420
...,...,...,...,...,...,...,...,...,...,...,...,...
86,86,12.868306,10.540350,0.324373,505.687873,2818.985742,12.335084,0.379605,591.792747,0,2922.395610,3420
87,87,1.139535,0.007319,0.000225,0.351157,0.021401,12.335084,0.379605,591.792747,0,2.029353,3420
88,88,33.746260,48.476608,1.491840,2325.732385,29158.988754,12.335084,0.379605,591.792747,0,13440.524236,3420
89,89,1.395894,0.013454,0.000414,0.645470,0.032506,12.335084,0.379605,591.792747,0,3.730201,3420


# Ahora os toca a vosotros :)

In [ ]:
msToKnots = np.float64(1.943844)

##  Speed

In [ ]:
# Ind
def myCalculateSpeed(coords, prevCoords, timeDiff, speedMultiplier=2):
    # Assume that 1º = 111,139 meters
    # Multiply the obtained speed by the speedMultiplier (2)
    # Returns speed in knots <---- KNOTS, OJO!
    return("The earth is flat")

In [ ]:
def myCalculateAcc(timeDiff, speed):
    acc = np.zeros(len(speed))
    # Some calculations here :)
    return(acc)

In [ ]:
def myCalculateSpeedAccPandas(df):
    speed = np.zeros(len(df))
    timeDiff = np.zeros(len(df))
    for i in range(1, len(df)):
        timeDiff[i] = (df.iloc[i]['timestamp'] - df.iloc[i-1]['timestamp']).total_seconds()
        speed[i] = myCalculateSpeed(df.iloc[i], df.iloc[i-1], timeDiff[i])
    acc = myCalculateAcc(timeDiff, speed)
    return(pd.DataFrame({'id':list(range(0,len(df))),'speed': speed, 'acc': acc, 'timeDiff': timeDiff}))

## Emisiones

In [ ]:
# Individual samples
def myTransientPowerME(VTransient, VDesign, PInstalled, EpsilonP=0.8,
                     VSafety=0.5):
    return(0)

In [ ]:
# Ind
def myTransientPowerAE(speed, shipType, instPow):
    return(0)

In [ ]:
def myCalcSOxEmissionFactor(SFOC=200, SC=0.001):
    return(0)

In [ ]:
def myCalcCO2EmissionFactor(SFOC=200, CC=0.85):
    return(0)

In [ ]:
def myCalcNOxEmissionFactor(rpm):
    return(0)

In [ ]:
def myCalcABC(speedDiff, timeDiff, alpha=582):
    # speed is in knots, we need it in m/s here!
    # np.abs gives us the absolute value of a number
    return(33)

In [ ]:
def myCalcCOEmissionFactor(speedDiff, timeDiff, COBase = 0.8):
    return(1337)

In [ ]:
def myComputeEmissions(ship, speed, time):
    # Static for every step
    NOxEF = myCalcNOxEmissionFactor(ship['rpm'])
    SOxEF = myCalcSOxEmissionFactor()
    CO2EF = myCalcCO2EmissionFactor()

    # Dinamic regarding speed! (Vector!)
    # Duplicate first element
    s = pd.concat([pd.Series(speed.iloc[0]), speed])
    t = pd.concat([pd.Series(time.iloc[0]), time])


    speedDiff = np.diff(s, n=1)
    timeDiff =  np.diff(t, n=1)
    timeDiff = [ max(1, int(t.total_seconds())) for t in timeDiff ]

    #COEF = calcCOEmissionFactor(speed, prevSpeed, timeDiff)
    COEF = np.array([ myCalcCOEmissionFactor(s, t) for s,t in zip(speedDiff, timeDiff) ])

    pME = np.array([ myTransientPowerME(s, ship['designSpeed'], ship['installedME']) for s in speed ])
    pAE = np.array([ myTransientPowerAE(s, ship['type'], ship['installedAE']) for s in speed ])

    # Dividing by 3600 makes the emissions from hours to seconds
    emis = pd.DataFrame({
        'id': list(range(0,len(speed))),
        'speed': speed,
        'noxme': (NOxEF * pME / 3600),
        'soxme': (SOxEF * pME / 3600),
        'co2me': (CO2EF * pME / 3600),
        'come': (COEF * pME / 3600),
        'noxae': (NOxEF * pAE / 3600),
        'soxae': (SOxEF * pAE / 3600),
        'co2ae': (CO2EF * pAE / 3600),
        'coae': 0, # (COEF * pAE / 3600),
        'pme': pME,
        'pae': pAE
    })
    return(emis)

In [ ]:
def myGetShipProfile():
    ship = dict({
        'type': 'Passenger/Ro-Ro Cargo Ship',
        'designSpeed': 21.4,
        'installedME': 18006,
        'installedAE': 3420,
        'rpm': 500
    })
    return(ship)

## Pruebas de velocidad

In [ ]:
speed = mcalculateSpeedAccPandas(trace1)

In [ ]:
myspeed = myCalculateSpeedAccPandas(trace1)

## Pruebas de emisiones

In [ ]:
emis = computeEmissions(ship, speed['speed'], trace1['timestamp'])
myemis = myComputeEmissions(ship, myspeed['speed'], trace1['timestamp'])